In [4]:
import os
import sys
import pandas
import numpy
import librosa
import torch
import s3prl
from zipfile import ZipFile
import glob
import csv
import soundfile as sf
from subprocess import Popen, PIPE
from os.path import dirname
import os
import math
import numpy as np
import random
import sed_eval
import dcase_util
import pickle
from tensorflow.keras import regularizers
from keras.regularizers import l2

In [5]:
# !apt-get install sox --quiet

In [6]:
source_training = []
# with open('./clean/synthetic_source_test.txt', 'r') as f:
#     print(f.readlines()[0][:-1])

In [7]:
def read_synthetic_txt(folder: str, subset: str, setup='source', remove_newline = False):
    path = f"/notebooks/VOICe/{folder}/synthetic_{setup}_{subset}.txt"
    with open(path, 'r') as f:
        li = [ p[:-1] if remove_newline else p for p in f.readlines()]
    return li

In [8]:
# clean_source_training_names = read_synthetic_txt('9dB', 'training', remove_newline = True)
# clean_source_validation_names = read_synthetic_txt('9dB', 'validation', remove_newline = True)
# clean_source_test_names = read_synthetic_txt('9dB', 'test', remove_newline = True)

In [9]:
def read_annotation(filename):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in spamreader:
            events.append(row)
    return events

In [10]:
events = read_annotation("./9dB/vehicle_207.txt")

In [11]:
envs = ['vehicle', 'outdoor', 'indoor']
vehicle_train_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/vehicle_source_training.txt")]
indoor_train_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/indoor_source_training.txt")]
outdoor_train_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/outdoor_source_training.txt")]
train_files = vehicle_train_files + indoor_train_files + outdoor_train_files

vehicle_test_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/vehicle_source_test.txt")]
indoor_test_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/indoor_source_test.txt")]
outdoor_test_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/outdoor_source_test.txt")]
test_files = vehicle_test_files + indoor_test_files + outdoor_test_files

vehicle_validation_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/vehicle_source_validation.txt")]
indoor_validation_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/indoor_source_validation.txt")]
outdoor_validation_files = ["./9dB/" + p[0] for p in read_annotation("./9dB/outdoor_source_validation.txt")]
validation_files = vehicle_validation_files + indoor_validation_files + outdoor_validation_files


In [12]:
mono_vehicle_train_files = [p.replace("9dB", "9dB-mono") for p in vehicle_train_files]
mono_indoor_train_files = [p.replace("9dB", "9dB-mono") for p in indoor_train_files]
mono_outdoor_train_files = [p.replace("9dB", "9dB-mono") for p in outdoor_train_files]
mono_train_files = mono_vehicle_train_files + mono_indoor_train_files + mono_outdoor_train_files

mono_vehicle_test_files = [p.replace("9dB", "9dB-mono") for p in vehicle_test_files]
mono_indoor_test_files = [p.replace("9dB", "9dB-mono") for p in indoor_test_files]
mono_outdoor_test_files = [p.replace("9dB", "9dB-mono") for p in outdoor_test_files]
mono_test_files = mono_vehicle_test_files + mono_indoor_test_files + mono_outdoor_test_files

mono_vehicle_validation_files = [p.replace("9dB", "9dB-mono") for p in vehicle_validation_files]
mono_indoor_validation_files = [p.replace("9dB", "9dB-mono") for p in indoor_validation_files]
mono_outdoor_validation_files = [p.replace("9dB", "9dB-mono") for p in outdoor_validation_files]
mono_validation_files = mono_vehicle_validation_files + mono_indoor_validation_files + mono_outdoor_validation_files

In [10]:
# os.makedirs(dirname(train_files[0]).replace("9dB", "9dB-mono"))

In [11]:
# len(glob.glob("./9dB-mono/*.wav"))

In [12]:
# # conversion to mono
# %%time
# for sound in train_files+test_files+validation_files:
#   temp_file = sound.replace("9dB", "9dB-mono")
#   command = command = "sox " + sound + " " + temp_file + " channels 1"
#   p = Popen(command, stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
#   output, err = p.communicate()

In [13]:
train_files_mono = [p.replace("9dB", "9dB-mono") for p in train_files]

In [14]:
%%time
test_files_mono = [p.replace("9dB", "9dB-mono") for p in test_files]

CPU times: user 84 µs, sys: 35 µs, total: 119 µs
Wall time: 125 µs


In [15]:
%%time
validation_files_mono = [p.replace("9dB", "9dB-mono") for p in validation_files]

CPU times: user 49 µs, sys: 20 µs, total: 69 µs
Wall time: 74.9 µs


In [16]:
vehicle_train_events = []
for audio in vehicle_train_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    vehicle_train_events += tmp
    
vehicle_test_events = []
for audio in vehicle_test_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    vehicle_test_events += tmp
    
vehicle_validation_events = []
for audio in vehicle_validation_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    vehicle_validation_events += tmp
    
indoor_train_events = []
for audio in indoor_train_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    indoor_train_events += tmp
    
indoor_test_events = []
for audio in indoor_test_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    indoor_test_events += tmp
    
indoor_validation_events = []
for audio in indoor_validation_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    indoor_validation_events += tmp

outdoor_train_events = []
for audio in outdoor_train_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    outdoor_train_events += tmp
    
outdoor_test_events = []
for audio in outdoor_test_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    outdoor_test_events += tmp
    
outdoor_validation_events = []
for audio in outdoor_validation_files:
    path = audio.replace("wav", "txt")
    tmp = [ [audio] + ann for ann in read_annotation(path)]
    outdoor_validation_events += tmp

In [17]:
print(len(vehicle_train_events))
print(len(vehicle_test_events))
print(len(vehicle_validation_events))
print("")
print(len(indoor_train_events))
print(len(indoor_test_events))
print(len(indoor_validation_events))
print("")
print(len(outdoor_train_events))
print(len(outdoor_test_events))
print(len(outdoor_validation_events))

7539
2448
2545

8016
2594
2663

8013
2572
2631


In [18]:
def construct_examples(audio_path, win_len = 2.56, hop_len = 1.0, sr = 44100.0):

  win_len_t = win_len
  hop_len_t = hop_len

  win_len = int(sr*win_len)
  hop_len = int(sr*hop_len)

  a, sr = sf.read(audio_path)

  if a.shape[0] < win_len:
    a_padded = np.zeros((win_len, ))
    a_padded[0:a.shape[0]] = a  

  else:
    no_of_hops = math.ceil((a.shape[0] - win_len) / hop_len)
    a_padded = np.zeros((int(win_len + hop_len*no_of_hops), ))
    a_padded[0:a.shape[0]] = a  

  a_ex = [a_padded[i - win_len : i] for i in range(win_len, a_padded.shape[0]+1, hop_len)]
  win_ranges = [((i - win_len)/sr, i/sr) for i in range(win_len, a_padded.shape[0]+1, hop_len)]

  return a_ex, win_ranges #chunks large audio file into windows, and gives the start and end time of each window

In [19]:
def construct_labels(annotation_path, win_start, win_end, win_len):
  """
    cycling through all the annotations and modifying them if they entire or any part of the old annotation is in the current focus window
  """
  events = read_annotation(annotation_path)
  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]

  curr_ann = []

  for a in ann:
    #   win_start = 1
    #   win_end = 2
    #   a = [1.1, 1.2, baby]
    #   a = [1.9, 2.2, baby]
    #   a = [0.9, 2.2, baby]
    if a[1] > win_start and a[0] < win_end: # checking if it exceeds the window
    # if a[0] >= win_start and a[0] < win_end: => this will check only if the audio starts within the window
      curr_start = max(a[0] - win_start, 0.0)
      curr_end = min(a[1] - win_start, win_len)
      curr_ann.append([curr_start, curr_end, a[2]])    

  class_set = set([c[2] for c in curr_ann]) # obtaining the list of all unique anns in the current focus window
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in curr_ann:
    class_wise_events[c[2]].append(c)

    # grouping all annotations by their class
    # {
    #     "baby":[
    #             [1.1, 1.3, 'baby'],
    #             [1.2, 1.4, 'baby'],
    #             [0.1, 1.2, 'baby'],
    #             [1.1, 2.2, 'baby'],
    #     ],
    #     "gun":[
    #             [1.1, 1.2, 'baby'],
    #             [0.1, 1.2, 'baby'],
    #             [1.1, 2.2, 'baby'],
    #     ],
    #     ....
    # }
    
  max_event_silence = 0.0
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1: #skipping the last ann in that class to compare ann[i] and ann[i+1]
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1]) #merging two annotations for the same time period into 1
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events
    # all events is corrected dictionary in the form of 2d list, removing distinc
        #     all_events = [
    #             [1.1, 1.3, 'baby'],
    #             [1.2, 1.4, 'baby'],
    #             [0.1, 1.2, 'baby'],
    #             [1.1, 2.2, 'baby'],
    #                   ...
    #             [1.1, 1.2, 'gun'],
    #             [0.1, 1.2, 'gun'],
    #             [1.1, 2.2, 'gun'],
    #                   ...
    #             [1.1, 1.2, 'breaking'],
    #             [0.1, 1.2, 'breaking'],
    #             [1.1, 2.2, 'breaking'],
    #     ],
  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])
#   sorted all events by their start time, so can be possible that ann -> baby, gun, 

  return all_events

In [20]:
class_dict = {
    "babycry" : 0,
    "gunshot" : 1,
    "glassbreak" : 2
}
rev_class_dict = ["babycry", "gunshot", "glassbreak"]

In [21]:
def to_seg_by_class(events, class_dict, hop_len = 441, n_frames = 257, sr=44100):
  """
        converting annotations to one hot vectors for the classes, mulitlabel
  """
# events = smoothe_events(events)
#   hop_len_t = 441/(441*100) = 0.01s
#   each frame is at a gap of 10ms here in each window
# therefore 257*10ms = 2570ms = 2.57s = length of each audio window/frame 
# more finegrained labels, at every 10ms gap in each window
  labels = np.zeros((n_frames, 3), dtype=np.float32)

  for e in events:
    t1 = float(e[0])
    t1 = int(t1 / hop_len * sr) # frame number beginning
    t2 = float(e[1])
    t2 = int(t2 / hop_len * sr) # frame number ending

    labels[t1:t2, class_dict[e[2]]] = 1    
  
  return labels 

In [22]:
def get_universal_labels(events, class_dict, ex_length = 10.0, no_of_div = 32):
#   its generating output formatted for for neural network 
# REMEMBER: bins are disjoint sequences, frames can be overlapping
# win_length here is of the bin, and not the frame window length we took above
  win_length = ex_length/no_of_div
  labels = np.zeros((no_of_div, len(class_dict.keys()) * 3))
  
  for e in events:

    start_time = float(e[0])
    stop_time = float(e[1])

    start_bin = int(start_time // win_length)
    stop_bin = int(stop_time // win_length)

    start_time_2 = start_time - start_bin * win_length
    stop_time_2 = stop_time - stop_bin * win_length

    n_bins = stop_bin - start_bin

    if n_bins == 0:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, stop_time_2]    

    elif n_bins == 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

    elif n_bins > 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      for i in range(1, n_bins):
        labels[start_bin + i, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

  # labels[:, [1, 2, 4, 5]] /= win_length

  for i in range(len(labels)):
    for j in range(len(labels[i])):
      if j % 3 != 0:
        labels[i][j] /= win_length

  return labels

In [23]:
# os.makedirs("./9dB-mono/train-data/vehicle")
# os.makedirs("./9dB-mono/train-data/indoor")
# os.makedirs("./9dB-mono/train-data/outdoor")

# os.makedirs("./9dB-mono/test-data/vehicle")
# os.makedirs("./9dB-mono/test-data/indoor")
# os.makedirs("./9dB-mono/test-data/outdoor")

# os.makedirs("./9dB-mono/validation-data/vehicle")
# os.makedirs("./9dB-mono/validation-data/indoor")
# os.makedirs("./9dB-mono/validation-data/outdoor")


In [24]:
envs = ["vehicle", "indoor", "outdoor"]

In [25]:
%%time
"""
Construct train set
"""

win_len = 2.56
hop_len = 1.96
a_ex_train = { k:[] for k in envs }
a_labels_train = { k:[] for k in envs}

# win_size = 10.0
# win_start = 16.0
# win_end = win_start + win_size
for e in envs:
  for i, audio in enumerate(eval(f"{e}_train_files")):
    a, win_ranges = construct_examples(audio.replace("9dB", "9dB-mono"), win_len=win_len, hop_len=hop_len)
    a_ex_train[e] += a
    for w in win_ranges: 
      labels_t = construct_labels(audio.replace(".wav", ".txt"), w[0], w[1], win_len=win_len)
      ll = get_universal_labels(labels_t, class_dict, ex_length=win_len, no_of_div = 9)
      # ll = to_seg_by_class(labels_t, class_dict) # ll is the multihot vector at every 10ms gap in each window
      a_labels_train[e].append(ll)
      # a_labels_train.append(to_seg_by_class(labels_t, class_dict))

CPU times: user 7.64 s, sys: 18 s, total: 25.7 s
Wall time: 10min 36s


In [25]:
sr = 44100
audio_duration_s = 7938000/sr
num_windows = (audio_duration_s * sr - win_len*sr)/(hop_len*sr)
num_windows

NameError: name 'win_len' is not defined

In [25]:
def get_log_melspectrogram(audio, sr = 44100, hop_length = 441, win_length = 1764, n_fft = 2048, n_mels = 40, fmin = 0, fmax = 22050):
    """Return the log-scaled Mel bands of an audio signal."""
    audio_2 = librosa.util.normalize(audio)
    bands = librosa.feature.melspectrogram(
        y=audio_2, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels)
    return librosa.core.power_to_db(bands)

In [28]:
# a_ex_train

In [29]:
# M = get_log_melspectrogram(a_ex_train["vehicle"][0])
# M.shape

In [30]:
%%time
for e in envs:
  for i, a in enumerate(a_ex_train[e]):
    M = get_log_melspectrogram(a).T #(257, 40)??why transpose
    np.save(f"./9dB-mono/train-data/{e}/ex-" + str(i) + ".npy", M)

CPU times: user 13min 30s, sys: 37min 20s, total: 50min 50s
Wall time: 13min 39s


In [31]:
%%time
for e in envs:
  for i, a in enumerate(a_labels_train[e]):
    np.save(f"./9dB-mono/train-data/{e}/label-" + str(i) + ".npy", a)

CPU times: user 4.88 s, sys: 1.59 s, total: 6.47 s
Wall time: 1min 7s


In [32]:
%%time
"""
Construct validation set
"""

win_len = 2.56
hop_len = 1.96
a_ex_validation = { k:[] for k in envs }
a_labels_validation = { k:[] for k in envs}

# win_size = 10.0
# win_start = 16.0
# win_end = win_start + win_size
for e in envs:
  for i, audio in enumerate(eval(f"{e}_validation_files")):
    a, win_ranges = construct_examples(audio.replace("9dB", "9dB-mono"), win_len=win_len, hop_len=hop_len)
    a_ex_validation[e] += a
    for w in win_ranges: 
      labels_t = construct_labels(audio.replace(".wav", ".txt"), w[0], w[1], win_len=win_len)
      ll = get_universal_labels(labels_t, class_dict, ex_length=win_len, no_of_div = 9)
      # ll = to_seg_by_class(labels_t, class_dict) # ll is the multihot vector at every 10ms gap in each window
      a_labels_validation[e].append(ll)
      # a_labels_validation.append(to_seg_by_class(labels_t, class_dict))

CPU times: user 2.18 s, sys: 5.39 s, total: 7.56 s
Wall time: 4min 19s


In [33]:
%%time
for e in envs:
  for i, a in enumerate(a_ex_validation[e]):
    M = get_log_melspectrogram(a).T #(257, 40)??why transpose
    np.save(f"./9dB-mono/validation-data/{e}/ex-" + str(i) + ".npy", M)

CPU times: user 4min 39s, sys: 12min 58s, total: 17min 38s
Wall time: 4min 45s


In [34]:
%%time
for e in envs:
  for i, a in enumerate(a_labels_validation[e]):
    np.save(f"./9dB-mono/validation-data/{e}/label-" + str(i) + ".npy", a)

CPU times: user 1.65 s, sys: 904 ms, total: 2.55 s
Wall time: 21.9 s


In [50]:
# a_labels_validation

In [51]:
%%time
"""
Construct test set
"""

win_len = 2.56
hop_len = 1.96
a_ex_test = { k:[] for k in envs }
a_labels_test = { k:[] for k in envs}

# win_size = 10.0
# win_start = 16.0
# win_end = win_start + win_size
for e in envs:
  for i, audio in enumerate(eval(f"{e}_test_files")):
    a, win_ranges = construct_examples(audio.replace("9dB", "9dB-mono"), win_len=win_len, hop_len=hop_len)
    a_ex_test[e] += a
    for w in win_ranges: 
      labels_t = construct_labels(audio.replace("wav", "txt"), w[0], w[1], win_len=win_len)
      ll = get_universal_labels(labels_t, class_dict, ex_length=win_len, no_of_div = 9)
      # ll = to_seg_by_class(labels_t, class_dict) # ll is the multihot vector at every 10ms gap in each window
      a_labels_test[e].append(ll)
      # a_labels_test.append(to_seg_by_class(labels_t, class_dict))

CPU times: user 1.99 s, sys: 3.8 s, total: 5.79 s
Wall time: 5.81 s


In [52]:
%%time
for e in envs:
  for i, a in enumerate(a_ex_test[e]):
    M = get_log_melspectrogram(a).T #(257, 40)??why transpose
    np.save(f"./9dB-mono/test-data/{e}/ex-" + str(i) + ".npy", M)

CPU times: user 4min 46s, sys: 13min 30s, total: 18min 16s
Wall time: 4min 55s


In [53]:
%%time
for e in envs:
  for i, a in enumerate(a_labels_test[e]):
    np.save(f"./9dB-mono/test-data/{e}/label-" + str(i) + ".npy", a)

CPU times: user 1.79 s, sys: 931 ms, total: 2.72 s
Wall time: 31.8 s


In [26]:
import re

def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [27]:
train_examples = { k:[] for k in envs }
for e in envs:
    data = glob.glob(f"./9dB-mono/train-data/{e}/ex-*.npy") # + glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True) 
    #data = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True) 
    sort_nicely(data)

    labels = glob.glob(f"./9dB-mono/train-data/{e}/label-*.npy") #+ glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
    #labels = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
    sort_nicely(labels)

    train_examples[e] = [(data[i], labels[i]) for i in range(len(data))]

    random.seed(4)
    random.shuffle(train_examples[e])

In [28]:
partition = {}
partition['train'] = train_examples
for e in envs:
    random.shuffle(partition['train'][e])

In [29]:
partition.keys()

dict_keys(['train'])

In [30]:
validation_examples = { k:[] for k in envs }
for e in envs:
    data = glob.glob(f"./9dB-mono/validation-data/{e}/ex-*.npy")
    sort_nicely(data)

    labels = glob.glob(f"./9dB-mono/validation-data/{e}/label-*.npy")
    sort_nicely(labels)

    validation_examples[e] = [(data[i], labels[i]) for i in range(len(data))]

    random.seed(4)
    random.shuffle(validation_examples[e])


In [31]:
# validation_examples

In [32]:
partition['validation'] = validation_examples
for e in envs:
    random.shuffle(partition['validation'][e])

In [33]:
test_examples = { k:[] for k in envs }
for e in envs:
    data = glob.glob(f"./9dB-mono/test-data/{e}/ex-*.npy")
    sort_nicely(data)

    labels = glob.glob(f"./9dB-mono/test-data/{e}/label-*.npy")
    sort_nicely(labels)

    test_examples[e] = [(data[i], labels[i]) for i in range(len(data))]

    random.seed(4)
    random.shuffle(test_examples[e])


In [34]:
partition['test'] = test_examples
for e in envs:
    random.shuffle(partition['test'][e])

In [35]:
# # %%quiet
# !git clone https://github.com/DemisEom/SpecAugment.git
# !pip install ./SpecAugment/
# !pip install tensorflow-addons

In [36]:
from SpecAugment import spec_augment_tensorflow

In [37]:
# !pip install tensorflow-io --quiet

In [38]:
import tensorflow_io as tfio
import tensorflow as tf
import keras

In [39]:
import tensorflow as tf
import keras

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_examples, batch_size=128, epoch_size = 16384, dim=(1, ),
                 n_classes=2, shuffle=True):
        'Initialization'
        print("Constructor called!!!")
        self.dim = dim
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        self.list_examples = list_examples
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        #print("The self.list_examples is {}".format(self.list_examples))
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
        
    def on_epoch_end(self):
      self.indexes = np.arange(len(self.list_examples))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # # Initialization
        X = np.empty([self.batch_size, 257, 40, 1], dtype=np.float64)
        y = np.empty([self.batch_size, 9, 9], dtype=np.float64)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
          # Store sample

          xx = np.load(ID[0])

          X[i, :, :, 0] = xx

          # Store class
          yy = np.load(ID[1])
          # yy2 = yy[:, [1, 2, 4, 5]]
          y[i, :, :] = yy

        tau = X.shape[1]          
        v = X.shape[2]

        warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
        warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)

        X = warped_frequency_time_sepctrogram



        return X, y

In [71]:
# Parametersa
params = {'dim': (1, ),
          'batch_size': 32,
          'epoch_size': 0,
          'n_classes': 2,
          'shuffle': True}

# Generators
training_generator  = { k:[] for k in envs }
validation_generator  = { k:[] for k in envs }
test_generator  = { k:[] for k in envs }
for e in envs:
    training_generator[e] = DataGenerator(partition['train'][e], **params)
    validation_generator[e] = DataGenerator(partition['validation'][e], **params)
    test_generator[e] = DataGenerator(partition['test'][e], **params)


Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!
Constructor called!!!


In [72]:
# X, y = training_generator['vehicle'].__getitem__(0)
# y.shape

In [40]:
def my_loss_fn(y_true, y_pred):
  weight = tf.constant([1.0])
  squared_difference = tf.square(y_true - y_pred)

  ss_True = squared_difference[:, :, 0] * 0 + 1 # (5, 5, 18) => (5, 5), all values 1
  ss_0 = y_true[:, :, 0]# (5, 5)
  ss_1 = y_true[:, :, 3]# (5, 5)
  ss_2 = y_true[:, :, 6]# (5, 5)
  sss = tf.stack((ss_True, ss_0, ss_0, ss_True, ss_1, ss_1,
                  ss_True, ss_2, ss_2,), axis = 2)
# y_pred/true (5, 5, 18):
# [
#  [
#   [prob001, start001, end001, prob002, start002, end002, prob003, start003, end003, prob004, start004, end004, prob005, start005, end005, prob006, start006, end006],
#   [prob011, start011, end011, prob012, start012, end012, prob013, start013, end013, prob014, start014, end014, prob015, start015, end015, prob016, start016, end016],
#   [prob021, start021, end021, prob022, start022, end022, prob023, start023, end023, prob024, start024, end024, prob025, start025, end025, prob026, start026, end026],
#   [prob031, start031, end031, prob032, start032, end032, prob033, start033, end033, prob034, start034, end034, prob035, start035, end035, prob036, start036, end036],
#   [prob041, start041, end041, prob042, start042, end042, prob043, start043, end043, prob044, start044, end044, prob045, start045, end045, prob046, start046, end046],
#  ],
#     [...],
#     [...],
#     [...],
#     [...]
# ]

# sss:
# [
#  [
#   [1, prob001, prob001, 1, prob002, prob002, ...., 1, prob006, prob006], 
#   [1, prob011, prob011, 1, prob0 12, prob0 12, ...., 1, prob016, prob016],
#   [1, prob021, prob021, 1, prob002, prob0022 ...., 1, prob0062 prob0062],
#   [1, prob031, prob031, 1, prob003, prob0023 ...., 1, prob0063 prob0063],
#   [1, prob041, prob041, 1, prob004, prob0024 ...., 1, prob0064 prob0064],
#  ],
#  [...],
#  [...],
#  [...],
#  [...]
# ]
  squared_difference =  tf.multiply(squared_difference, sss) #element wise multiplication
  return tf.reduce_sum(squared_difference, axis=[-1, -2])  # Note the `axis=-1`

In [33]:
# a = tf.random.normal((5, 5, 9))
# b = tf.random.normal((5, 5, 9))
# my_loss_fn(a, b)

In [41]:
class MyCustomCallback_3(tf.keras.callbacks.Callback):
  def __init__(self, model_dir, patience=0):
    super(MyCustomCallback_3, self).__init__()
    self.patience = patience
    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None
    self.model_best_path = os.path.join(model_dir, 'model-best.h5')
    self.model_last_path = os.path.join(model_dir, 'model-last-epoch.h5')
    self.custom_params = {"best_loss":np.inf, "last_epoch":0}
    
    self.custom_params_path = os.path.join(model_dir, 'custom_params.pickle')
    if os.path.isfile(self.custom_params_path):
      with open(self.custom_params_path, 'rb') as f:
        self.custom_params = pickle.load(f)

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best F1 as 0.0.
    self.is_impatient = False

  def on_train_end(self, logs=None):
    if not self.is_impatient:
      print("Restoring model weights from the end of the best epoch.")
      self.model.set_weights(self.best_weights)
      # temp_model_path = self.model_path.replace(".h5", "_temp.h5")
      #os.remove(temp_model_path)

  def on_epoch_end(self, epoch, logs=None):
    current_val_loss = logs.get("val_loss")
    self.model.save_weights(self.model_last_path)
    self.custom_params["last_epoch"] = self.custom_params["last_epoch"] + 1

    if current_val_loss < self.custom_params['best_loss']:
      self.custom_params['best_loss'] = current_val_loss
      self.wait = 0
      self.best_weights = self.model.get_weights()
      self.model.save_weights(self.model_best_path)

    else:
        self.wait += 1
        if self.wait >= self.patience:
            self.stopped_epoch = epoch
            self.is_impatient = True
            self.model.stop_training = True
            print("Restoring model weights from the end of the best epoch.")
            self.model.set_weights(self.best_weights)
            #os.remove(temp_model_path)
    with open(self.custom_params_path, 'wb') as f:
      pickle.dump(self.custom_params, f, pickle.HIGHEST_PROTOCOL)

In [42]:
# list(enumerate(mono_vehicle_validation_files))

In [77]:
%%time
# Creates mel spctrograms for training

win_length = 2.56
hop_size = 1.96
mss_ins = { k:[] for k in envs }
win_ranges_list = { k:[] for k in envs } # for each audio, generate all the windows, and then append a matrixsingle melspectrogram for all the windows

for e in envs:
  for ii, audio in enumerate(eval(f"mono_{e}_validation_files")):
    a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)

    mss_in = np.zeros((len(a), 257, 40))

    preds = np.zeros((len(a), 9, 9)) # why 9?? is it because in yolo there are 9 subimages?
  #   I think each input spectrogram to the model for a particular window is being split into 9 'bins', like YOLO?


    for i in range(len(a)):
      M = get_log_melspectrogram(a[i]) # (40, 257)
      mss_in[i, :, :] = M.T # (257, 40)
    mss_ins[e].append(mss_in)
    win_ranges_list[e].append(win_ranges)
#   mss_ins : [
#     [
#      [...spectrogram of window1],
#      [...spectrogram of window2],
#      [...spectrogram of window3],
#     ] (audio1),
#     [
#      [...spectrogram of window1],
#      [...spectrogram of window2],
#      [...spectrogram of window3],
#     ] (audio2),
#       ...
# ]



CPU times: user 4min 15s, sys: 11min 23s, total: 15min 39s
Wall time: 7min 29s


In [78]:
# mss_ins['vehicle'][0].shape

In [43]:
mss_ins = 1
win_ranges_list = 2

In [44]:
def mk_preds_YOHO_mel(model, ind, env, mss_ins=mss_ins, win_ranges_list=win_ranges_list, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, max_event_silence = 0.3, sampling_rate = 44100):
  """
    ind: int, indicating which audio file's windows to predict on
  """
#   print(ind)
  preds = model.predict(mss_ins[env][ind])
#   print(preds)
  events = []

  for i in range(len(preds)): # cycling through predictions for each window of current audio
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)): # cycling from [0, 9) ? To traverse through each subwindow?
      for jjj in range(0, 3): # jjj traverses through classes?
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges_list[env][ind][i][0] # is this start time of the window?
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)
    
  
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1: #merging anns if reqd
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events

In [45]:
def frames_to_time(f, sr = 44100.0, hop_size = 441):
  return f * hop_size / sr

def preds_to_se(p, win_start, audio_clip_length = 2.56):
  start_dicts = [-100] * 9
  stop_dicts = [-100] * 9
#   print(p)
  audio_events = []

  n_frames = p.shape[0]

  for j in range(0, p.shape[1], 3):
    if p[0, j] >= 0.5:
      start_dicts[j] = 0

  for j in range(0, p.shape[1], 3):
    for i in range(n_frames - 1):
      if p[i, j] < 0.5 and p[i+1, j] >= 0.5:
        start_dicts[j] = i+1

      elif p[i, j] >= 0.5 and p[i + 1, j] < 0.5:
        stop_dicts[j] = i
        start_time = frames_to_time(start_dicts[j])
        stop_time = frames_to_time(stop_dicts[j])

        audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j//3]])
        start_dicts[j] = -100
        stop_dicts[j] = -100

    if start_dicts[j] != -100:
      start_time = frames_to_time(start_dicts[j])
      stop_time = audio_clip_length
      audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j//3]])
      start_dicts[j] = -100
      stop_dicts[j] = -100

  audio_events.sort(key = lambda x: x[0]) 
  return audio_events

In [75]:
# a, win_ranges = construct_examples("./clean-mono/synthetic_008.wav", win_len=2.56,hop_len=1.96)
a, win_ranges = construct_examples("./9dB-mono/outdoor_008.wav", win_len=2.56,hop_len=1.96)

max_event_silence = 0.3
preds = np.zeros((len(a), 9, 9))
mss_in = np.zeros((len(a), 257, 40))

for i in range(len(a)):
  M = get_log_melspectrogram(a[i])
  mss_in[i, :, :] = M.T

preds = model_vehicle.predict(mss_in)
events = []

for i in range(len(preds)):
  p = preds[i, :, :]
  events_curr = []
  # win_width = win_length / no_of_div

  events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

  events += events_curr

print(events)

class_set = set([c[2] for c in events])
class_wise_events = {}

for c in list(class_set):
  class_wise_events[c] = []


for c in events:
  class_wise_events[c[2]].append(c)
  

all_events = []

for k in list(class_wise_events.keys()):
  curr_events = class_wise_events[k]
  count = 0

  while count < len(curr_events) - 1:
    if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
      curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
      del curr_events[count + 1]
    else:
      count += 1

  all_events += curr_events

for i in range(len(all_events)):
  all_events[i][0] = round(all_events[i][0], 3)
  all_events[i][1] = round(all_events[i][1], 3)

all_events.sort(key=lambda x: x[0])

print(all_events)


[[0.0, 0.01, 'gunshot'], [0.07, 2.56, 'gunshot'], [1.96, 2.02, 'gunshot'], [2.02, 4.52, 'glassbreak'], [3.92, 3.9699999999999998, 'glassbreak'], [3.94, 3.9699999999999998, 'gunshot'], [3.96, 6.48, 'babycry'], [5.88, 5.93, 'babycry'], [5.96, 8.44, 'gunshot'], [7.85, 10.4, 'gunshot'], [7.88, 7.8999999999999995, 'glassbreak'], [9.8, 9.8, 'gunshot'], [9.860000000000001, 12.360000000000001, 'glassbreak'], [11.76, 11.82, 'glassbreak'], [11.77, 14.32, 'babycry'], [11.799999999999999, 14.32, 'gunshot'], [13.72, 13.75, 'babycry'], [13.74, 13.780000000000001, 'gunshot'], [13.780000000000001, 16.28, 'glassbreak'], [15.68, 15.69, 'glassbreak'], [17.66, 17.7, 'babycry'], [17.68, 20.2, 'glassbreak'], [19.6, 19.64, 'glassbreak'], [19.630000000000003, 19.650000000000002, 'gunshot'], [21.59, 24.119999999999997, 'babycry'], [21.61, 24.119999999999997, 'gunshot'], [23.52, 23.56, 'gunshot'], [23.599999999999998, 26.08, 'glassbreak'], [25.490000000000002, 25.53, 'glassbreak'], [25.51, 25.53, 'gunshot'], [2

In [76]:
a, win_ranges = construct_examples("./9dB-mono/outdoor_008.wav", win_len=2.56,hop_len=1.96)

max_event_silence = 0.3
preds = np.zeros((len(a), 9, 9))
mss_in = np.zeros((len(a), 257, 40))

for i in range(len(a)):
  M = get_log_melspectrogram(a[i])
  mss_in[i, :, :] = M.T

preds = model_indoor.predict(mss_in)
# print(preds)
events = []

for i in range(len(preds)):
  p = preds[i, :, :]
  events_curr = []
  # win_width = win_length / no_of_div

  events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

  events += events_curr

print(events)

class_set = set([c[2] for c in events])
class_wise_events = {}

for c in list(class_set):
  class_wise_events[c] = []


for c in events:
  class_wise_events[c[2]].append(c)
  

all_events = []

for k in list(class_wise_events.keys()):
  curr_events = class_wise_events[k]
  count = 0

  while count < len(curr_events) - 1:
    if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
      curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
      del curr_events[count + 1]
    else:
      count += 1

  all_events += curr_events

for i in range(len(all_events)):
  all_events[i][0] = round(all_events[i][0], 3)
  all_events[i][1] = round(all_events[i][1], 3)

all_events.sort(key=lambda x: x[0])

print(all_events)


[[0.06, 2.56, 'gunshot'], [1.96, 1.99, 'babycry'], [1.96, 2.01, 'gunshot'], [2.03, 4.52, 'glassbreak'], [3.92, 3.96, 'glassbreak'], [3.94, 3.9899999999999998, 'gunshot'], [3.9499999999999997, 6.48, 'babycry'], [5.88, 5.93, 'babycry'], [5.96, 8.44, 'gunshot'], [7.859999999999999, 7.8999999999999995, 'gunshot'], [7.88, 7.8999999999999995, 'glassbreak'], [9.860000000000001, 12.360000000000001, 'glassbreak'], [11.76, 14.32, 'babycry'], [11.76, 11.83, 'glassbreak'], [11.799999999999999, 14.32, 'gunshot'], [13.72, 13.76, 'babycry'], [13.74, 13.790000000000001, 'gunshot'], [13.790000000000001, 16.28, 'glassbreak'], [15.68, 15.7, 'glassbreak'], [17.650000000000002, 17.7, 'babycry'], [17.68, 20.2, 'glassbreak'], [19.6, 19.650000000000002, 'glassbreak'], [21.56, 24.119999999999997, 'babycry'], [21.61, 24.119999999999997, 'gunshot'], [21.61, 21.61, 'glassbreak'], [23.52, 23.55, 'babycry'], [23.53, 23.55, 'gunshot'], [23.599999999999998, 26.08, 'glassbreak'], [25.490000000000002, 25.52, 'glassbrea

In [42]:
# model['outdoor'].weights

In [46]:
def extract_labels_2(annotation_path, env, data_type="eval"):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]
  if env=="clean":
    n_label = f"./clean-mono/{data_type}-files-2/" + os.path.basename(annotation_path)
  else:    
    n_label = f"./9dB-mono/{data_type}-files-2/{env}/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [85]:
# for e in envs:
#     os.makedirs(f"./9dB-mono/eval-files-2/{e}")

In [86]:
for e in envs:
    for audio in eval(f"{e}_validation_files"):
      extract_labels_2(audio.replace(".wav", ".txt"), e)

In [87]:
# vehicle_validation_files
for e in envs:
#     os.makedirs(f"./9dB-mono/test-files-2/{e}")
    for audio in eval(f"{e}_test_files"):
      extract_labels_2(audio.replace(".wav", ".txt"), e, "test")

In [105]:
for audio in eval(f"clean_test_files"):
    extract_labels_2(audio.replace(".wav", ".txt"), "clean", "test")

In [88]:
# vehicle_validation_files
for e in envs:
#     os.makedirs(f"./9dB-mono/train-files-2/{e}")
    for audio in eval(f"{e}_train_files"):
      extract_labels_2(audio.replace(".wav", ".txt"), e, "train")

In [89]:
# vehicle_train_files

In [47]:
class MyCustomCallback_44(tf.keras.callbacks.Callback):
  """
    callback when validating/testing?
  """
  def __init__(self, env):
    super(MyCustomCallback_44, self).__init__()
    self.best_f1 = 0.0
    self.best_error = np.inf
    self.env = env
    
  def on_train_begin(self, logs=None):
    pass

  def on_train_end(self, logs=None):
    pass

  def on_epoch_end(self, epoch, logs=None):
    if epoch > 1:
      for ii, audio in enumerate(eval(f"mono_{self.env}_validation_files")):
        audio_file_path = audio
        see = mk_preds_YOHO_mel(self.model, ii, self.env)
        n_label = n_label = f"./9dB-mono/eval-files-2/{self.env}/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

        with open(n_label, 'w') as fp:
          fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

      destination = f"./9dB-mono/eval-files-2/{self.env}/"
      test_set = glob.glob(destination + "*[0-9].txt")

      eval_path = "./9dB-mono/"


      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt")
          }
          for tt in test_set
      ]

      data = []

      # Get used event labels
      all_data = dcase_util.containers.MetaDataContainer()
      for file_pair in file_list:
          reference_event_list = sed_eval.io.load_event_list(
              filename=file_pair['reference_file']
          )
          estimated_event_list = sed_eval.io.load_event_list(
              filename=file_pair['estimated_file']
          )

          data.append({'reference_event_list': reference_event_list,
                      'estimated_event_list': estimated_event_list})

          all_data += reference_event_list

      event_labels = all_data.unique_event_labels

      # Start evaluating

      # Create metrics classes, define parameters
      segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
          event_label_list=event_labels,
          time_resolution=1.0
      )

      event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
          event_label_list=event_labels,
          t_collar=1.0
      )

      # Go through files
      for file_pair in data:
          segment_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

          event_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

      # Get only certain metrics
      overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
      curr_f1 = overall_segment_based_metrics['f_measure']['f_measure']
      curr_error = overall_segment_based_metrics['error_rate']['error_rate']

      if curr_f1 > self.best_f1:
        self.best_f1 = curr_f1
#         self.model.save_weights(f"./9dB-mono/model-{self.env}-best-f1.h5")

      if curr_error < self.best_error:
        self.best_error = curr_error
#         self.model.save_weights(f"./9dB-mono/model-{self.env}-best-error.h5")

      print("F-measure: {:.3f} vs {:.3f}".format(curr_f1, self.best_f1))
      print("Error rate: {:.3f} vs {:.3f}".format(curr_error, self.best_error))

      # Or print all metrics as reports

In [48]:
LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    ([3, 3], 1,   64),
    ([3, 3], 2,  128),
    ([3, 3], 1,  128),
    ([3, 3], 2,  256),
    ([3, 3], 1,  256),
    ([3, 3], 2,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 2, 1024),
    ([3, 3], 1, 1024),
    ([3, 3], 1, 512),
    ([3, 3], 1, 256),
    ([3, 3], 1, 128),
    # ([3, 3], 1, 128),
    # ([3, 3], 1, 128)
]

In [49]:
"""
Manually define YamNet
"""

# params = yamnet_params.Params()
def ret_model(e):
    m_features = tf.keras.Input(shape=(257, 40), name="mel_input")
    X = m_features
    X = tf.keras.layers.Reshape((257, 40, 1))(X)
    X = tf.keras.layers.Conv2D(filters = 32, kernel_size=[3, 3], strides=2, padding='same', use_bias=False,
                               activation=None, name = "layer1/conv",
                                 kernel_regularizer=l2(1e-3), bias_regularizer=l2(1e-3))(X)
    X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer1/bn")(X)
    X = tf.keras.layers.ReLU(name="layer1/relu")(X)

    # X = tf.keras.layers.SpatialDropout2D(0.5)(X)

    for i in range(len(LAYER_DEFS)):
      X = tf.keras.layers.DepthwiseConv2D(kernel_size=LAYER_DEFS[i][0], strides = LAYER_DEFS[i][1], depth_multiplier=1, padding='same', use_bias=False,
                                          activation=None, name="layer"+ str(i + 2)+"/depthwise_conv")(X)
      X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/depthwise_conv/bn")(X)
      X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/depthwise_conv/relu")(X)
      X = tf.keras.layers.Conv2D(filters = LAYER_DEFS[i][2], kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None,
                                 name = "layer"+ str(i + 2)+"/pointwise_conv",
                                 kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(X)
      X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/pointwise_conv/bn")(X)
      X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/pointwise_conv/relu")(X)

      X = tf.keras.layers.SpatialDropout2D(0.1)(X)

    _, _, sx, sy = X.shape
    X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)
    pred = tf.keras.layers.Conv1D(9,kernel_size=1, activation="sigmoid")(X)
    
    return tf.keras.Model(
          name=f"yamnet_{e}", inputs=m_features,
          outputs=[pred])
# model = tf.keras.Model(
#       name='yamnet_frames', inputs=m_features,
#       outputs=[pred])

In [50]:
# model_vehicle = tf.keras.Model(
#       name='yamnet_frames', inputs=m_features,
#       outputs=[pred])

In [51]:
# model_vehicle.summary()

In [52]:
# model_vehicle.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn)

In [53]:
# validation_generator

In [54]:
# # model_vehicle.fit(training_generator['vehicle'], validation_data=validation_generator['vehicle'], epochs=1000, callbacks=[MyCustomCallback_44('vehicle'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1, patience=3)], verbose=1)
# model_vehicle.fit(x=training_generator['vehicle'], 
#                   validation_data=validation_generator['vehicle'], 
#                   epochs=1000, 
#                   callbacks=[MyCustomCallback_44('vehicle'), 
#                              tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                                               min_delta=1, patience=3)], 
#                   verbose=1)

In [55]:
# validation_generator['vehicle'].__getitem__(0)[1].shape

In [50]:
model_vehicle = ret_model('vehicle')
#     print(model[e].weights)
model_vehicle.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn)
#     model_vehicle.fit(training_generator['vehicle'], validation_data=validation_generator['vehicle'], epochs=1000, callbacks=[MyCustomCallback_44('vehicle'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1, patience=3)], verbose=1)
# model_vehicle.fit(x=training_generator['vehicle'], 
#                       validation_data=validation_generator['vehicle'], 
#                       epochs=1000, 
#                       callbacks=[MyCustomCallback_44('vehicle'), 
#                                  tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                                                   min_delta=0.1, patience=5)], 
#                       verbose=1)

In [64]:
model_vehicle.summary()

Model: "yamnet_vehicle"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel_input (InputLayer)       [(None, 257, 40)]         0         
_________________________________________________________________
reshape (Reshape)            (None, 257, 40, 1)        0         
_________________________________________________________________
layer1/conv (Conv2D)         (None, 129, 20, 32)       288       
_________________________________________________________________
layer1/bn (BatchNormalizatio (None, 129, 20, 32)       96        
_________________________________________________________________
layer1/relu (ReLU)           (None, 129, 20, 32)       0         
_________________________________________________________________
layer2/depthwise_conv (Depth (None, 129, 20, 32)       288       
_________________________________________________________________
layer2/depthwise_conv/bn (Ba (None, 129, 20, 32)    

In [51]:
model_outdoor = ret_model('outdoor')
#     print(model[e].weights)
model_outdoor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn)
#     model_vehicle.fit(training_generator['vehicle'], validation_data=validation_generator['vehicle'], epochs=1000, callbacks=[MyCustomCallback_44('vehicle'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1, patience=3)], verbose=1)
# model_outdoor.fit(x=training_generator['outdoor'], 
#                       validation_data=validation_generator['outdoor'], 
#                       epochs=1000, 
#                       callbacks=[MyCustomCallback_44('outdoor'), 
#                                  tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                                                   min_delta=0.1, patience=5)], 
#                       verbose=1)

In [52]:
model_indoor = ret_model('indoor')
#     print(model[e].weights)
model_indoor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn)
#     model_vehicle.fit(training_generator['vehicle'], validation_data=validation_generator['vehicle'], epochs=1000, callbacks=[MyCustomCallback_44('vehicle'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1, patience=3)], verbose=1)
# model_indoor.fit(x=training_generator['indoor'], 
#                       validation_data=validation_generator['indoor'], 
#                       epochs=1000, 
#                       callbacks=[MyCustomCallback_44('indoor'), 
#                                  tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                                                   min_delta=0.1, patience=5)], 
#                       verbose=1)

In [53]:
for e in envs:
    print(f"./9dB-mono/model-{e}-best-f1.h5")
#     eval(f"model_{e}").load_weights(f"./9dB-mono/model-{e}-best-f1.h5")
#     eval(f"model_{e}").save_weights(f"./9dB-mono/YOHO-{e}-best-weights.h5")
    eval(f"model_{e}").load_weights(f"./9dB-mono/YOHO-{e}-best-weights.h5")
#     model[e].load_weights(f"./9dB-mono/YOHO-{e}-best-weights.h5")


./9dB-mono/model-vehicle-best-f1.h5
./9dB-mono/model-indoor-best-f1.h5
./9dB-mono/model-outdoor-best-f1.h5


In [54]:
model_vehicle.weights[0]

<tf.Variable 'layer1/conv/kernel:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
array([[[[-0.03945932,  0.05435377, -0.06127225, -0.1240996 ,
          -0.06548793, -0.10602772,  0.01179217,  0.15351444,
          -0.0708956 ,  0.00302531, -0.01958794, -0.07131065,
          -0.12859735,  0.01545038,  0.03563861, -0.09651648,
           0.1059859 ,  0.01661397,  0.16023912, -0.04712846,
          -0.1316442 ,  0.06491029, -0.05699551,  0.11205365,
          -0.0340417 , -0.03233761, -0.00112153, -0.00186365,
          -0.02604831, -0.1491993 ,  0.01351031, -0.14023542]],

        [[ 0.10563325,  0.09492715, -0.20141101,  0.01233308,
           0.07064327, -0.0462825 , -0.10420957, -0.03865984,
          -0.11487799,  0.06946927,  0.00101797, -0.15788592,
          -0.00805537, -0.00691957, -0.07113483, -0.03979451,
          -0.12259253,  0.20050098, -0.01974095, -0.15876664,
           0.07889087, -0.10178389, -0.00084766,  0.0495701 ,
           0.09450669, -0.09637129, -0.12265382,  0

In [55]:
model_outdoor.weights[0]

<tf.Variable 'layer1/conv/kernel:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
array([[[[-0.1244984 ,  0.11777262, -0.18859343,  0.11526607,
          -0.1365871 ,  0.1307396 , -0.13624492, -0.02853931,
          -0.09755321,  0.04807639, -0.09991184, -0.02648628,
           0.06905626, -0.11273265,  0.00149452, -0.12980641,
          -0.02627374, -0.08476586, -0.11629748, -0.05491259,
           0.06145082, -0.00595863, -0.0665348 , -0.1847171 ,
          -0.0893825 ,  0.0173008 , -0.00201564,  0.04492539,
          -0.00607836, -0.03624047, -0.07244577, -0.03653622]],

        [[ 0.04241982, -0.08019183,  0.0434911 ,  0.11245901,
           0.05650868, -0.10565972,  0.01911126,  0.065647  ,
           0.14892153,  0.06850974, -0.01886157, -0.14977741,
          -0.12894708,  0.0818271 , -0.09053322, -0.04110025,
          -0.27125064,  0.04655409,  0.00543232,  0.10267838,
          -0.14965798,  0.04355825, -0.08884573,  0.10151968,
           0.12243472,  0.06552087,  0.13589726,  0

In [56]:
def smoothe_events(events):

  ann = events

  curr_ann = ann

  class_set = set([c[2] for c in curr_ann])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in curr_ann:
    class_wise_events[c[2]].append(c)
    
  max_event_silence = 1.0
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events

In [57]:
model_clean = ret_model('clean') 
model_clean.load_weights(f"./clean-mono/YOHO-best-weights-2.h5")


In [ ]:
# model['indoor'].name

In [ ]:
len(mono_outdoor_train_files)

In [65]:
# %%time
# # Creates mel spctrograms for training

# win_length = 2.56
# hop_size = 1.96
# mss_ins_train = { k:[] for k in envs }
# win_ranges_list_train = { k:[] for k in envs } # for each audio, generate all the windows, and then append a matrixsingle melspectrogram for all the windows
# ctr = 0
# for e in envs:
#   for ii, audio in enumerate(eval(f"mono_{e}_train_files")):
#     a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)
#     print(f'{ctr}; ', end='')
#     ctr+=1
#     mss_in = np.zeros((len(a), 257, 40))

#     preds = np.zeros((len(a), 9, 9)) # why 9?? is it because in yolo there are 9 subimages?
#   #   I think each input spectrogram to the model for a particular window is being split into 9 'bins', like YOLO?


#     for i in range(len(a)):
#       M = get_log_melspectrogram(a[i]) # (40, 257)
#       mss_in[i, :, :] = M.T # (257, 40)
#     mss_ins_train[e].append(mss_in)
#     win_ranges_list_train[e].append(win_ranges)

In [85]:
%%time
# Creates mel spctrograms for validation

win_length = 2.56
hop_size = 1.96
mss_ins_validation = { k:[] for k in envs }
win_ranges_list_val = { k:[] for k in envs } # for each audio, generate all the windows, and then append a matrixsingle melspectrogram for all the windows

for e in envs:
  for ii, audio in enumerate(eval(f"mono_{e}_validation_files")):
    a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)
#     print(len(a))
    mss_in = np.zeros((len(a), 257, 40))

    preds = np.zeros((len(a), 9, 9)) # why 9?? is it because in yolo there are 9 subimages?
  #   I think each input spectrogram to the model for a particular window is being split into 9 'bins', like YOLO?


    for i in range(len(a)):
      M = get_log_melspectrogram(a[i]) # (40, 257)
      mss_in[i, :, :] = M.T # (257, 40)
    mss_ins_validation[e].append(mss_in)
    win_ranges_list_val[e].append(win_ranges)

KeyboardInterrupt: 

In [58]:
clean_test_files = [ p[0] for p in read_annotation('./clean/synthetic_source_test.txt')]
mono_clean_test_files = ['./clean-mono/'+p for p in clean_test_files]
clean_test_files = ['./clean/'+p for p in clean_test_files]
mono_clean_test_files

['./clean-mono/synthetic_139.wav',
 './clean-mono/synthetic_140.wav',
 './clean-mono/synthetic_141.wav',
 './clean-mono/synthetic_142.wav',
 './clean-mono/synthetic_143.wav',
 './clean-mono/synthetic_144.wav',
 './clean-mono/synthetic_145.wav',
 './clean-mono/synthetic_146.wav',
 './clean-mono/synthetic_147.wav',
 './clean-mono/synthetic_148.wav',
 './clean-mono/synthetic_149.wav',
 './clean-mono/synthetic_150.wav',
 './clean-mono/synthetic_151.wav',
 './clean-mono/synthetic_152.wav',
 './clean-mono/synthetic_153.wav',
 './clean-mono/synthetic_154.wav',
 './clean-mono/synthetic_155.wav',
 './clean-mono/synthetic_156.wav',
 './clean-mono/synthetic_157.wav',
 './clean-mono/synthetic_158.wav',
 './clean-mono/synthetic_159.wav',
 './clean-mono/synthetic_160.wav',
 './clean-mono/synthetic_161.wav',
 './clean-mono/synthetic_162.wav',
 './clean-mono/synthetic_163.wav',
 './clean-mono/synthetic_164.wav',
 './clean-mono/synthetic_165.wav',
 './clean-mono/synthetic_166.wav',
 './clean-mono/synth

In [ ]:
clean_test_files

In [59]:
%%time
# Creates mel spctrograms for testing

win_length = 2.56
hop_size = 1.96
mss_ins_test = { k:[] for k in envs+["clean"] }
win_ranges_list_test = { k:[] for k in envs+["clean"] } # for each audio, generate all the windows, and then append a matrixsingle melspectrogram for all the windows

for e in envs+["clean"]:
  for ii, audio in enumerate(eval(f"mono_{e}_test_files")):
    a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)
#     print(len(a))
    mss_in = np.zeros((len(a), 257, 40))

    preds = np.zeros((len(a), 9, 9)) # why 9?? is it because in yolo there are 9 subimages?
  #   I think each input spectrogram to the model for a particular window is being split into 9 'bins', like YOLO?


    for i in range(len(a)):
      M = get_log_melspectrogram(a[i]) # (40, 257)
      mss_in[i, :, :] = M.T # (257, 40)
    mss_ins_test[e].append(mss_in)
    win_ranges_list_test[e].append(win_ranges)

CPU times: user 6min 41s, sys: 16min 57s, total: 23min 38s
Wall time: 10min 39s


In [60]:
# mono_outdoor_test_files

In [61]:
def report_f1(model, src_env, target_env, mss_ins1, win_ranges_list1, type_data = "test"):
  best_f1 = np.inf * -1
  best_error = np.inf
  print(f"mono_{target_env}_{type_data}_files")
#   print(model.name)
#   print(model.weights[0])
  tmp1 = "9dB"
  if target_env == "clean":
    tmp1 = "clean"
  for ii, audio in enumerate(eval(f"mono_{target_env}_{type_data}_files")):
      audio_file_path = audio
      see = mk_preds_YOHO_mel(model, ii, target_env, mss_ins1, win_ranges_list1)
      path = f"./{tmp1}-mono/{type_data}-src-{src_env}-target-{target_env}/"
            
      try:
            os.makedirs(path)
      except Exception as e:
#             print(e)
            pass
      n_label = n_label = path + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

      with open(n_label, 'w') as fp:
          fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))
        
  if type_data == "validation":
      destination = f"./{tmp1}-mono/eval-files-2/{target_env}/"
      tmp = 'eval'
  else:
      if tmp1 == "clean":
          destination = f"./{tmp1}-mono/{type_data}-files-2/"
      else:    
          destination = f"./{tmp1}-mono/{type_data}-files-2/{target_env}/"
      tmp = type_data
    
  test_set = glob.glob(destination + "*[0-9].txt")

  eval_path = "./9dB-mono/"
  
  if tmp1 == "clean":
      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt").replace(
              f"{tmp}-files-2/",
                      f"{type_data}-src-{src_env}-target-{target_env}/")
          }
          for tt in test_set
      ]
  else:
      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt").replace(
              f"{tmp}-files-2/{target_env}/",
                      f"{type_data}-src-{src_env}-target-{target_env}/")
          }
          for tt in test_set
      ]
  data = []

  # Get used event labels
  all_data = dcase_util.containers.MetaDataContainer()
  for file_pair in file_list:
      reference_event_list = sed_eval.io.load_event_list(
          filename=file_pair['reference_file']
      )
      estimated_event_list = sed_eval.io.load_event_list(
          filename=file_pair['estimated_file']
      )
      data.append({'reference_event_list': reference_event_list,
                  'estimated_event_list': estimated_event_list})

      all_data += reference_event_list
#       print(data[0]['reference_event_list'][0])
#       print(data[0]['estimated_event_list'][0])
  event_labels = all_data.unique_event_labels

      # Start evaluating

  # Create metrics classes, define parameters
  segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
      event_label_list=event_labels,
      time_resolution=1.0
  )

  event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
      event_label_list=event_labels,
      t_collar=1.0
  )
  # Go through files
  for file_pair in data:
      segment_based_metrics.evaluate(
          reference_event_list=file_pair['reference_event_list'],
          estimated_event_list=file_pair['estimated_event_list']
      )

      event_based_metrics.evaluate(
          reference_event_list=file_pair['reference_event_list'],
          estimated_event_list=file_pair['estimated_event_list']
      )

  # Get only certain metrics
  overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
  curr_f1 = overall_segment_based_metrics['f_measure']['f_measure']
  curr_error = overall_segment_based_metrics['error_rate']['error_rate']
  if curr_f1 > best_f1:
    best_f1 = curr_f1
#         self.model.save_weights(f"./9dB-mono/model-{self.env}-best-f1.h5")

  if curr_error < best_error:
    best_error = curr_error
#         self.model.save_weights(f"./9dB-mono/model-{self.env}-best-error.h5")

  print("F-measure: {:.3f} vs {:.3f}".format(curr_f1, best_f1))
  print("Error rate: {:.3f} vs {:.3f}".format(curr_error, best_error))

      # Or print all metrics as reports
  return (best_f1, best_error)

In [88]:
report_f1(model_vehicle, 'vehicle', 'indoor', mss_ins_test, win_ranges_list_test, type_data = "test")

mono_indoor_test_files
F-measure: 0.747 vs 0.747
Error rate: 0.438 vs 0.438


(0.747067174773967, 0.43772023900720086)

In [89]:
report_f1(model_indoor, 'indoor', 'indoor', mss_ins_test, win_ranges_list_test, type_data = "test")

mono_indoor_test_files
F-measure: 0.803 vs 0.803
Error rate: 0.383 vs 0.383


(0.8034310221586848, 0.38333077983759767)

In [62]:
src = envs+["clean"]
target = src
print(f"For SNR 9dB")
all_results2 = {}
for s in src:
    for t in target:
        print(f"{s}/{t}")
#         print(eval(f"{t}_test_files")
#         if s=="clean":
#               m = model_clean
#         else:
#               m = model[s]
        all_results2[f"{s}/{t}"] = report_f1(eval(f"model_{s}"), s, t, mss_ins_test, win_ranges_list_test, type_data = "test")
#         need to load test target data

For SNR 9dB
vehicle/vehicle
mono_vehicle_test_files
F-measure: 0.834 vs 0.834
Error rate: 0.309 vs 0.309
vehicle/indoor
mono_indoor_test_files
F-measure: 0.747 vs 0.747
Error rate: 0.438 vs 0.438
vehicle/outdoor
mono_outdoor_test_files
F-measure: 0.774 vs 0.774
Error rate: 0.401 vs 0.401
vehicle/clean
mono_clean_test_files
F-measure: 0.847 vs 0.847
Error rate: 0.290 vs 0.290
indoor/vehicle
mono_vehicle_test_files
F-measure: 0.830 vs 0.830
Error rate: 0.348 vs 0.348
indoor/indoor
mono_indoor_test_files
F-measure: 0.803 vs 0.803
Error rate: 0.383 vs 0.383
indoor/outdoor
mono_outdoor_test_files
F-measure: 0.820 vs 0.820
Error rate: 0.350 vs 0.350
indoor/clean
mono_clean_test_files
F-measure: 0.812 vs 0.812
Error rate: 0.422 vs 0.422
outdoor/vehicle
mono_vehicle_test_files
F-measure: 0.827 vs 0.827
Error rate: 0.326 vs 0.326
outdoor/indoor
mono_indoor_test_files
F-measure: 0.774 vs 0.774
Error rate: 0.396 vs 0.396
outdoor/outdoor
mono_outdoor_test_files
F-measure: 0.806 vs 0.806
Error rate

In [63]:
all_results2

{'vehicle/vehicle': (0.8344041117892708, 0.30862181580666226),
 'vehicle/indoor': (0.747067174773967, 0.43772023900720086),
 'vehicle/outdoor': (0.773843984264826, 0.401386748844376),
 'vehicle/clean': (0.8468541700536498, 0.29045571797076525),
 'indoor/vehicle': (0.8295625942684766, 0.3478118876551274),
 'indoor/indoor': (0.8034310221586848, 0.38333077983759767),
 'indoor/outdoor': (0.8202353973243659, 0.3496147919876733),
 'indoor/clean': (0.8122695767585901, 0.4223559759243336),
 'outdoor/vehicle': (0.8273449920508744, 0.325930764206401),
 'outdoor/indoor': (0.7738601823708208, 0.3962003983453347),
 'outdoor/outdoor': (0.8059952588514185, 0.3499229583975347),
 'outdoor/clean': (0.8225496476617553, 0.34496990541702494),
 'clean/vehicle': (0.7554055915163201, 0.5249836708033965),
 'clean/indoor': (0.7268402756601817, 0.6355140186915889),
 'clean/outdoor': (0.7313566936208445, 0.5782742681047766),
 'clean/clean': (0.8647179909427748, 0.24333619948409285)}

In [90]:
src = ["clean"]+envs
target = envs
print(f"For SNR 9dB")
all_results = {}
for s in src:
    for t in target:
        print(f"{s}/{t}")
#         print(eval(f"{t}_test_files")
#         if s=="clean":
#               m = model_clean
#         else:
#               m = model[s]
        all_results[f"{s}/{t}"] = report_f1(eval(f"model_{s}"), s, t, mss_ins_test, win_ranges_list_test, type_data = "test")
#         need to load test target data

For SNR 9dB
clean/vehicle
mono_vehicle_test_files
F-measure: 0.759 vs 0.759
Error rate: 0.550 vs 0.550
clean/indoor
mono_indoor_test_files
F-measure: 0.727 vs 0.727
Error rate: 0.654 vs 0.654
clean/outdoor
mono_outdoor_test_files
F-measure: 0.742 vs 0.742
Error rate: 0.594 vs 0.594
vehicle/vehicle
mono_vehicle_test_files
F-measure: 0.834 vs 0.834
Error rate: 0.309 vs 0.309
vehicle/indoor
mono_indoor_test_files
F-measure: 0.747 vs 0.747
Error rate: 0.438 vs 0.438
vehicle/outdoor
mono_outdoor_test_files
F-measure: 0.774 vs 0.774
Error rate: 0.401 vs 0.401
indoor/vehicle
mono_vehicle_test_files
F-measure: 0.830 vs 0.830
Error rate: 0.348 vs 0.348
indoor/indoor
mono_indoor_test_files
F-measure: 0.803 vs 0.803
Error rate: 0.383 vs 0.383
indoor/outdoor
mono_outdoor_test_files
F-measure: 0.820 vs 0.820
Error rate: 0.350 vs 0.350
outdoor/vehicle
mono_vehicle_test_files
F-measure: 0.827 vs 0.827
Error rate: 0.326 vs 0.326
outdoor/indoor
mono_indoor_test_files
F-measure: 0.774 vs 0.774
Error rat

In [91]:
all_results

{'clean/vehicle': (0.7587086381636352, 0.5504572175048987),
 'clean/indoor': (0.7268690747629906, 0.6543588172207753),
 'clean/outdoor': (0.7417350236142182, 0.5936825885978428),
 'vehicle/vehicle': (0.8344041117892708, 0.30862181580666226),
 'vehicle/indoor': (0.747067174773967, 0.43772023900720086),
 'vehicle/outdoor': (0.773843984264826, 0.401386748844376),
 'indoor/vehicle': (0.8295625942684766, 0.3478118876551274),
 'indoor/indoor': (0.8034310221586848, 0.38333077983759767),
 'indoor/outdoor': (0.8202353973243659, 0.3496147919876733),
 'outdoor/vehicle': (0.8273449920508744, 0.325930764206401),
 'outdoor/indoor': (0.7738601823708208, 0.3962003983453347),
 'outdoor/outdoor': (0.8059952588514185, 0.3499229583975347)}

In [ ]:
# src = ["clean"]+envs
# target = envs
# print(f"For SNR 9dB")
# all_results = {}
# for s in src:
#     for t in target:
#         print(f"{s}/{t}")
# #         print(eval(f"{t}_test_files")
# #         if s=="clean":
# #               m = model_clean
# #         else:
# #               m = model[s]
#         all_results[f"{s}/{t}"] = report_f1(eval(f"model_{s}"), s, t, mss_ins_validation, win_ranges_list_val, type_data = "validation")
# #         need to load test target data


In [ ]:
# all_results

In [ ]:
# src = ["clean"]+envs
# target = envs
# print(f"For SNR 9dB")
# all_results = {}
# for s in src:
#     for t in target:
#         print(f"{s}/{t}")
# #         print(eval(f"{t}_test_files")
# #         if s=="clean":
# #               m = model_clean
# #         else:
# #               m = model[s]
#         all_results[f"{s}/{t}"] = report_f1(eval(f"model_{s}"), s, t, mss_ins_train, win_ranges_list_train, type_data = "train")
# #         need to load test target data


In [ ]:
# all_results